# **Creation of my dataset**

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI
import openai

In [ ]:
from google.colab import userdata
gpt_key = userdata.get('gptkey')
openai.api_key = gpt_key
client = OpenAI(api_key=gpt_key)

In [ ]:
chat_completion = client.chat.completions.create(
   messages=[
     {
         "role": "user",
         "content": "Je vais à",
     }
   ],
   model= "gpt-4o-mini",
)
print(chat_completion.choices[0].message.content)

Où vas-tu ? Je suis là pour t'aider si tu veux des recommandations ou des informations sur ta destination !


In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

# Function to scrape content from a given URL
def scrape_mitre_attack(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract text from paragraphs or other relevant elements
        paragraphs = soup.find_all('p')
        content = '\n'.join([p.get_text() for p in paragraphs])

        return url, content
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching {url}: {e}")
        return url, None

# Function to extract URLs from the index page
def get_urls_from_index(index_url):
    try:
        response = requests.get(index_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Example: Extracting all technique links from a list or table
        # You need to adjust the selector based on the actual HTML structure of the index page
        links = soup.find_all('a', href=True)  # Find all anchor tags with href attribute
        base_url = 'https://attack.mitre.org'
        urls = []

        for link in links:
            href = link['href']
            # Only add URLs that match the desired pattern, e.g., technique pages
            if href.startswith('/techniques/'):
                urls.append(base_url + href)

        return urls
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching {index_url}: {e}")
        return []

# Example usage
index_url = 'https://attack.mitre.org'  # Example index page for techniques
urls = get_urls_from_index(index_url)

# Scrape content from each URL
for url in urls:
    url, content = scrape_mitre_attack(url)
    if content:
        print(f"URL: {url}")
        print(f"Content: {content[:500]}...")  # Print the first 500 characters as a preview
        print("\n")
    else:
        print(f"No content found for {url}")


No content found for https://attack.mitre.org/techniques/
URL: https://attack.mitre.org/techniques/enterprise/
Content: 
                                        Techniques represent 'how' an adversary achieves a tactical goal by performing an action. For example, an adversary may dump credentials to achieve credential access.
                                    ...


URL: https://attack.mitre.org/techniques/mobile/
Content: 
                                        Techniques represent 'how' an adversary achieves a tactical goal by performing an action. For example, an adversary may dump credentials to achieve credential access.
                                    ...


URL: https://attack.mitre.org/techniques/ics/
Content: 
                                        Techniques represent 'how' an adversary achieves a tactical goal by performing an action. For example, an adversary may dump credentials to achieve credential access.
                                    ...


URL: https://atta

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import openai
import pandas as pd
# Function to scrape content from a given URL
def scrape_mitre_attack(url, max_length=1500):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract text from relevant elements
        elements = soup.find_all(['h1', 'h2', 'h3', 'p', 'li', 'table', 'code'])
        content = []

        for element in elements:
            if len(' '.join(content)) >= max_length:
                break

            if element.name == 'table':
                # Extract text from tables
                for row in element.find_all('tr'):
                    cells = row.find_all(['th', 'td'])
                    row_text = ' | '.join(cell.get_text().strip() for cell in cells)
                    content.append(row_text)
            elif element.name == 'code':
                # Extract code block content
                content.append(element.get_text().strip())
            else:
                # Extract text from headers, paragraphs, and list items
                content.append(element.get_text().strip())

        # Combine and truncate content if necessary
        combined_content = '\n'.join(content)
        if len(combined_content) > max_length:
            combined_content = combined_content[:max_length]

        return url, combined_content
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching {url}: {e}")
        return url, None
# Function to extract URLs from the index page
def get_urls_from_index(index_url):
    try:
        response = requests.get(index_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Example: Extracting all technique links from a list or table
        # You need to adjust the selector based on the actual HTML structure of the index page
        links = soup.find_all('a', href=True)  # Find all anchor tags with href attribute
        base_url = 'https://attack.mitre.org'
        urls = []

        for link in links:
            href = link['href']
            # Only add URLs that match the desired pattern, e.g., technique pages
            if href.startswith('/techniques/'):
                urls.append(base_url + href)

        return urls
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching {index_url}: {e}")
        return []

# Function to generate MCQs using OpenAI's API
def generate_mcqs(text):
    prompt = f"""
    You are a cybersecurity expert specializing in cyber threat intelligence. Given the text below, please
    generate 5 multiple-choice questions with four possible options each.

    Follow these requirements:
    1. Question Format: Each question must have four options. The options should be challenging and require
    careful consideration. Avoid creating options that could be interpreted as correct under different circumstances.
    2. Target Audience: The questions should be suitable for security professionals with three to five years of
    experience in cyber threat intelligence. Avoid generic questions such as “What is the objective?”, “Which
    operating system can be targeted?”.
    3. Content Coverage: Aim to cover various sections of the document to ensure a comprehensive evaluation of
    the candidate’s knowledge. Include context-specific questions that require an understanding of the document’s
    content.
    4. Technical Precision: Use precise terminology and concepts relevant to cyber threat intelligence. Incorporate
    situational or scenario-based questions where applicable.
    5. Include Technique IDs and Names: Ensure that all questions, where applicable, mention both the ID and the
    full name of the MITRE ATT&CK pattern technique.
    6. Premise Inclusion: Each question should include a premise indicating it pertains to MITRE ATT&CK,
    specifying the relevant platform (Enterprise, ICS, or Mobile) where necessary.
    7. Output Format: Return the output in this format (must be tab-separated):
    Question, Option A, Option B, Option C, Option D, Correct Answer (A, B, C, D).

    Important: Only return the content as specified. Do not include any additional text or commentary or labels like (Question, Option A, Option B, Option C, Option D, Correct Answer).

    Text:
    {text}
    """


    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.9,
            max_tokens=1500,
        )
        print(response.choices[0].message.content)
        return response.choices[0].message.content
    except openai.error.OpenAIError as e:
        print(f"An error occurred while generating MCQs: {e}")
        return None

def main():
    index_url = 'https://attack.mitre.org'
    all_mcqs = []
    max_mcqs = 1570
    total_mcqs = 0

    urls = get_urls_from_index(index_url)
    for url in urls:
        if total_mcqs >= max_mcqs:
            break
        url, content = scrape_mitre_attack(url)
        if content:
            mcq_output = generate_mcqs(content)
            if mcq_output:
                mcq_output = mcq_output.strip()
                questions_and_answers = mcq_output.split("\n\n")  # Splitting the output into individual questions and answers
                for qa in questions_and_answers:
                    if total_mcqs >= max_mcqs:
                        break
                    lines = qa.split('\n')
                    if len(lines) == 5:  # Ensure all parts are present
                        question = lines[0].strip()
                        option_a = lines[1].strip()
                        option_b = lines[2].strip()
                        option_c = lines[3].strip()
                        option_d = lines[4].strip()
                        correct_answer = lines[4][-1]  # Assuming the correct answer is mentioned at the end

                        # Verify the correct answer
                        if correct_answer not in ['A', 'B', 'C', 'D']:
                            print(f"Incorrect format for correct answer: {correct_answer} in {url}")
                            continue

                        all_mcqs.append([url, question, option_a, option_b, option_c, option_d, correct_answer])
                        total_mcqs += 1

    # Save all MCQs to a TSV file
    if all_mcqs:
        df = pd.DataFrame(all_mcqs, columns=['URL', 'Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Answer'])
        df.to_csv('/content/generated_mcqs.tsv', sep='\t', index=False)
        print(f"MCQs saved to generated_mcqs.tsv with {total_mcqs} questions.")

# Run the main function
if __name__ == "__main__":
    main()

In the context of MITRE ATT&CK Enterprise, what does technique T1548 specifically describe?	A) Bypassing network access controls	B) Manipulating web page content	C) Abusing elevation control mechanisms	D) Intercepting data transmission	C
Which sub-technique of T1548 in MITRE ATT&CK Enterprise involves exploiting application configurations to achieve higher privileges?	A) .002 Exploit Public-Facing Application	B) .001 Setuid and Setgid	C) .003 Path Interception	D) .004 Scheduled Task	B
According to MITRE ATT&CK Enterprise, which data source would be most appropriate to detect actions described in technique T1548?	A) Application logs	B) Network flow data	C) Authentication logs	D) Process monitoring	A
In the MITRE ATT&CK framework, which platform does technique T1548 apply to?	A) Mobile	B) Enterprise	C) ICS	D) Cloud	B
What is the primary tactical goal of an adversary when using technique T1548 as described in MITRE ATT&CK Enterprise section?	A) Data exfiltration	B) Credential access	C) Pr

AttributeError: module 'openai' has no attribute 'error'

In [ ]:
import csv

def add_prompt_column(tsv_file_path):
    # Read the data from the TSV file
    with open(tsv_file_path, 'r') as file:
        reader = csv.DictReader(file, delimiter='\t')
        rows = list(reader)

    # Define the prompt template without newlines
    prompt_template = (
        "You are a cybersecurity expert specializing in cyber threat intelligence. "
        "You are given a multiple-choice question (MCQ) from a Cyber Threat Intelligence (CTI) knowledge benchmark "
        "dataset. Your task is to choose the best option among the four provided. Return your answer as a "
        "single uppercase letter: A, B, C, or D. Question: {question} "
        "Options: A) {option_a} B) {option_b} C) {option_c} D) {option_d} "
        "Important: The last line of your answer should contain only the single letter corresponding to the best option, "
        "with no additional text."
    )

    # Add the 'Prompt' column to each row
    for row in rows:
        row['Prompt'] = prompt_template.format(
            question=row['Question'],
            option_a=row['Option A'],
            option_b=row['Option B'],
            option_c=row['Option C'],
            option_d=row['Option D']
        )

    # Write the updated data back to the TSV file
    output_file_path = tsv_file_path.replace('.tsv', '_gen_mcq.tsv')
    with open(output_file_path, 'w', newline='') as file:
        fieldnames = reader.fieldnames + ['Prompt']
        writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter='\t')

        writer.writeheader()
        writer.writerows(rows)

    return output_file_path


In [ ]:
path= add_prompt_column('/content/output.tsv')
print(path)

/content/output_gen_mcq.tsv


In [ ]:
!pip install transformers torch datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from datasets import load_dataset

# Define the path to your TSV file
data_files = {"cti_mcq": "/content/output_gen_mcq.tsv"}

# Load the dataset
dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

# Display some examples from the dataset
print(dataset['cti_mcq'][0])

{'Question': 'In the context of MITRE ATT&CK Enterprise, what does technique T1548 specifically describe?', 'Option A': 'Bypassing network access controls', 'Option B': 'Manipulating web page content', 'Option C': ' Abusing elevation control mechanisms', 'Option D': ' Intercepting data transmission', 'Prompt': 'You are a cybersecurity expert specializing in cyber threat intelligence. You are given a multiple-choice question (MCQ) from a Cyber Threat Intelligence (CTI) knowledge benchmark dataset. Your task is to choose the best option among the four provided. Return your answer as a single uppercase letter: A, B, C, or D. Question: In the context of MITRE ATT&CK Enterprise, what does technique T1548 specifically describe? Options: A) Bypassing network access controls B) Manipulating web page content C)  Abusing elevation control mechanisms D)  Intercepting data transmission Important: The last line of your answer should contain only the single letter corresponding to the best option, w

In [ ]:
dataset

DatasetDict({
    cti_mcq: Dataset({
        features: ['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Prompt', 'GT'],
        num_rows: 1500
    })
})

In [ ]:
from huggingface_hub import notebook_login
notebook_login ( )

In [ ]:
dataset.push_to_hub('cti_mcq')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Dorrajl/cti_mcq/commit/67808df55f649bbdb748ae2997d3f0dc8fbdec75', commit_message='Upload dataset', commit_description='', oid='67808df55f649bbdb748ae2997d3f0dc8fbdec75', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
pip install pypdf2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [7]:
import PyPDF2
import re
import csv

# Fonction pour extraire le texte d'un fichier PDF
def extract_text_from_pdf(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

# Fonction pour extraire les descriptions et CWE IDs à partir du texte
def extract_cwe_data(text):
    cve_cwe_pairs = []

    # Regex pour extraire le Weakness ID, titre, et description
    cwe_pattern = re.compile(r'(Weakness ID\s*:\s*(\d+)).*?\n(.*?)\nDescription(.*?)\nRelationships', re.DOTALL)

    matches = cwe_pattern.findall(text)
    for match in matches:
        weakness_id = match[1].strip()  # CWE ID (Weakness ID)
        cwe_title = match[2].strip()  # Titre du CWE (avant la description)
        cwe_description = match[3].strip().replace('\n', ' ')  # Description sans sauts de ligne

        # Créer la description complète à partir du titre et de la description
        full_description = f"{cwe_description}"

        # Construire le prompt personnalisé
        prompt = (f"You are a cybersecurity expert specializing in cyber threat intelligence. "
                  f"Analyze the following CVE description and map it to the appropriate CWE. "
                  f"Provide a brief justification for your choice. Ensure the last line of your response "
                  f"contains only the CWE ID.\nCVE Description: {full_description}")

        # Ajouter le tuple (description, Weakness ID, prompt) à la liste
        cve_cwe_pairs.append((f"CWE-{weakness_id}",full_description, prompt))

    return cve_cwe_pairs
def clean_text(text):
    return ' '.join(text.splitlines()).strip()
# Fonction pour écrire les résultats dans un fichier TSV
def write_to_tsv(data, output_file):
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        tsv_writer = csv.writer(file, delimiter='\t')
        tsv_writer.writerow(['GT','Description', 'Prompt'])  # En-tête du fichier

        for row in data:
            # Nettoyer chaque champ pour supprimer les retours à la ligne
            clean_row = [clean_text(cell) for cell in row]
            tsv_writer.writerow(clean_row)



# Extraire le texte du fichier PDF
text = extract_text_from_pdf("/content/cwe_v4.15.pdf")

# Extraire les descriptions CWE et Weakness ID
cve_cwe_data = extract_cwe_data(text)

# Nom du fichier TSV de sortie
output_file = 'resultat_cwe.tsv'

# Enregistrer les données dans un fichier TSV
write_to_tsv(cve_cwe_data, output_file)

print(f"Extraction terminée. Les résultats sont enregistrés dans {output_file}")


Extraction terminée. Les résultats sont enregistrés dans resultat_cwe.tsv


In [12]:
import PyPDF2
import re
import csv

# Fonction pour extraire le texte d'un fichier PDF
def extract_text_from_pdf(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

# Fonction pour extraire les descriptions et vecteurs CVSS
def extract_cvss_data(text):
    cvss_data = []

    # Regex pour capturer les descriptions sous CVE et les vecteurs CVSS
    cve_pattern = re.compile(r'(CVE-\d{4}-\d+)\n(.*?)(?=CVSS:3\.1)', re.DOTALL)
    cvss_pattern = re.compile(r'(CVSS:3\.1/.*?)\n', re.DOTALL)

    cve_matches = cve_pattern.findall(text)
    cvss_matches = cvss_pattern.findall(text)

    for i in range(min(len(cve_matches), len(cvss_matches))):
        cve_id = cve_matches[i][0].strip()
        description = cve_matches[i][1].strip().replace('\n', ' ')
        cvss_vector = cvss_matches[i].strip()

        # Créer le prompt personnalisé
        prompt =   prompt = (f"Analyze the following CVE description and calculate the CVSS v3.1 Base Score. "
                  f"Determine the values for each base metric: AV, AC, PR, UI, S, C, I, and A. "
                  f"Summarize each metric's value and provide the final CVSS v3.1 vector string. "
                  f"Valid options for each metric are as follows:\n"
                  f"- **Attack Vector (AV)**: Network (N), Adjacent (A), Local (L), Physical (P)\n"
                  f"- **Attack Complexity (AC)**: Low (L), High (H)\n"
                  f"- **Privileges Required (PR)**: None (N), Low (L), High (H)\n"
                  f"- **User Interaction (UI)**: None (N), Required (R)\n"
                  f"- **Scope (S)**: Unchanged (U), Changed (C)\n"
                  f"- **Confidentiality (C)**: None (N), Low (L), High (H)\n"
                  f"- **Integrity (I)**: None (N), Low (L), High (H)\n"
                  f"- **Availability (A)**: None (N), Low (L), High (H)\n"
                  f"Summarize each metric's value and provide the final CVSS v3.1 vector string.\n"
                  f"Ensure the final line of your response contains only the CVSS v3 Vector String "
                  f"in the following format:\n"
                  f"Example format: CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H\n"
                  f"CVE Description: {description}")

        # Ajouter le tuple (description, CVSS Vector, prompt) à la liste
        cvss_data.append((cve_id, description, cvss_vector, prompt))

    return cvss_data

def clean_text(text):
    return ' '.join(text.splitlines()).strip()

# Fonction pour écrire les résultats dans un fichier TSV
def write_to_tsv(data, output_file):
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        tsv_writer = csv.writer(file, delimiter='\t')
        tsv_writer.writerow(['CVE ID', 'Description', 'CVSS Vector', 'Prompt'])  # En-tête du fichier

        for row in data:
            # Nettoyer chaque champ pour supprimer les retours à la ligne
            clean_row = [clean_text(cell) for cell in row]
            tsv_writer.writerow(clean_row)

# Nom du fichier PDF à analyser
pdf_file = '/content/cvss-v40-examples.pdf'

# Extraire le texte du fichier PDF
text = extract_text_from_pdf(pdf_file)

# Extraire les descriptions et les vecteurs CVSS
cvss_data = extract_cvss_data(text)

# Nom du fichier TSV de sortie
output_file = 'resultat_cvss.tsv'

# Enregistrer les données dans un fichier TSV
write_to_tsv(cvss_data, output_file)

print(f"Extraction terminée. Les résultats sont enregistrés dans {output_file}")


Extraction terminée. Les résultats sont enregistrés dans resultat_cvss.tsv


In [15]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import pickle
import time

# Charger les dictionnaires alias et related
with open('/content/alias_dict.pickle', 'rb') as f:
    alias_dict = pickle.load(f)

with open('/content/related_dict.pickle', 'rb') as f:
    related_dict = pickle.load(f)


# URL de base pour Malpedia
base_url = "https://malpedia.caad.fkie.fraunhofer.de/actor/"


# Fonction pour remplacer les noms des acteurs, alias et malwares par [PLACEHOLDER]
def replace_placeholders_with_dicts(text, alias_dict, related_dict):
    for actor, aliases in alias_dict.items():
        text = re.sub(rf'\b{actor}\b', '[PLACEHOLDER]', text, flags=re.IGNORECASE)
        for alias in aliases:
            if alias != 'nan':
                text = re.sub(rf'\b{alias}\b', '[PLACEHOLDER]', text, flags=re.IGNORECASE)

    for actor, related in related_dict.items():
        text = re.sub(rf'\b{actor}\b', '[PLACEHOLDER]', text, flags=re.IGNORECASE)
        for rel in related:
            if rel != 'nan':
                text = re.sub(rf'\b{rel}\b', '[PLACEHOLDER]', text, flags=re.IGNORECASE)

    return text

# Fonction pour scraper une page individuelle d'un acteur (uniquement les paragraphes)
def scrape_report_page(url, session):
    response = session.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extraction des paragraphes
        paragraphs = soup.find_all('p')
        report_text = " ".join([p.get_text().strip() for p in paragraphs])  # Unir les paragraphes sur une seule ligne

        # Remplacer les noms sensibles par [PLACEHOLDER]
        cleaned_text = replace_placeholders_with_dicts(report_text, alias_dict, related_dict)

        # Génération du prompt
        prompt = f"You are given a threat report that describes a cyber incident. Any direct mentions of the threat actor group, specific campaign names, or malware names responsible have been replaced with [PLACEHOLDER]. Your task is to analyze the report and attribute the incident to a known threat actor based on the techniques, tactics, procedures (TTPs), and any other relevant information described. Please provide the name of the threat actor you believe is responsible and briefly explain your reasoning. Threat Report: {cleaned_text}"

        return cleaned_text, prompt
    else:
        print(f"Erreur {response.status_code} lors de la récupération des données de {url}")
        return None, None

# Fonction principale pour scraper toutes les pages et enregistrer dans un fichier TSV
def scrape_malpedia_to_tsv(output_file):
    session = requests.Session()

    # Ajouter l'en-tête User-Agent
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0'
    })

    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerow(['text', 'prompt'])

        # Pour chaque acteur dans alias_dict, construire l'URL et scraper la page
        for actor in alias_dict.keys():
            url = f"{base_url}{actor}"
            text, prompt = scrape_report_page(url, session)

            if text and prompt:
                # Nettoyer les retours à la ligne
                text = text.replace('\n', ' ').replace('\r', ' ').strip()
                prompt = prompt.replace('\n', ' ').replace('\r', ' ').strip()

                writer.writerow([text, prompt])
                print(f"Enregistré: {url}")

            # Ajout d'une pause entre les requêtes pour éviter le blocage
            time.sleep(2)

# Appel de la fonction principale avec le fichier de sortie
scrape_malpedia_to_tsv('malpedia_threat_reports.tsv')


Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/bahamut
Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/windshift
Erreur 404 lors de la récupération des données de https://malpedia.caad.fkie.fraunhofer.de/actor/confucius
Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/patchwork
Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/turla
Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/muddywater
Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/sandworm
Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/oilrig
Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/lazarus
Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/sidecopy
Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/transparent tribe
Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/mustang panda
Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/diamond sleet
Enregistré: https://malpedia.caad.fkie.fraunhofer.de/actor/mu